In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import tensorflow as tf
import os
import time
import re

2023-03-28 08:01:36.200949: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
jokes_data = pd.read_json('reddit_jokes.json')

In [3]:
jokes_data.shape

(194553, 4)

In [4]:
jokes_data['body'][:10]

0    Now I have to say "Leroy can you please paint ...
1    Pizza doesn't scream when you put it in the ov...
2    ...and being there really helped me learn abou...
3    A Sunday school teacher is concerned that his ...
4    He got caught trying to sell the two books to ...
5                             Because the p is silent.
6    To avoid leaving tracks like a snail while wal...
7    Apparently "Whatever's low in cholesterol" was...
8                                    Character actors!
9                        You follow his fresh prints. 
Name: body, dtype: object

In [5]:
jokes_data['title']

0          I hate how you cant even say black paint anymore
1         What's the difference between a Jew in Nazi Ge...
2                            I recently went to America....
3         Brian raises his hand and says, “He’s in Heaven.”
4         You hear about the University book store worke...
                                ...                        
194548       I like a girl with words tattooed on her back.
194549                                   I have sexdaily...
194550    What's the difference between a hippie chick a...
194551                                     new family robot
194552     I went to a zoo and there was only one animal...
Name: title, Length: 194553, dtype: object

In [6]:
len(jokes_data)

194553

In [7]:
def to_printable(s):
    return re.sub(r'[^\x00-\x7f]',r' ',s)   # Each char is a Unicode codepoint.

f = open('all_jokes.txt', 'w')
for i in range(len(jokes_data)):
    title = to_printable(jokes_data['title'][i])
    body = to_printable(jokes_data['body'][i].replace('\n', ' ').replace('\r', ''))

    f.write(f"{title}\n")
    f.write(f"{body}\n\n")

In [8]:
text = open('all_jokes.txt', 'rb').read().decode(encoding='utf-8')
print(len(text))

49118381


In [9]:
print(text[:250])

I hate how you cant even say black paint anymore
Now I have to say "Leroy can you please paint the fence?"

What's the difference between a Jew in Nazi Germany and pizza ?
Pizza doesn't scream when you put it in the oven .  I'm so sorry.

I recently 


In [10]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

100 unique characters


In [11]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab),
    mask_token=None
)

In [12]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(),
    invert=True,
    mask_token=None
)

In [13]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [14]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(49118381,), dtype=int64, numpy=array([46,  5, 77, ..., 19, 19,  3])>

In [15]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [16]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

I
 
h
a
t
e
 
h
o
w


2023-03-28 08:02:30.395319: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [49118381]
	 [[{{node Placeholder/_0}}]]


In [17]:
seq_length = 100

In [18]:
sequences = ids_dataset.batch(seq_length + 1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'I' b' ' b'h' b'a' b't' b'e' b' ' b'h' b'o' b'w' b' ' b'y' b'o' b'u'
 b' ' b'c' b'a' b'n' b't' b' ' b'e' b'v' b'e' b'n' b' ' b's' b'a' b'y'
 b' ' b'b' b'l' b'a' b'c' b'k' b' ' b'p' b'a' b'i' b'n' b't' b' ' b'a'
 b'n' b'y' b'm' b'o' b'r' b'e' b'\n' b'N' b'o' b'w' b' ' b'I' b' ' b'h'
 b'a' b'v' b'e' b' ' b't' b'o' b' ' b's' b'a' b'y' b' ' b'"' b'L' b'e'
 b'r' b'o' b'y' b' ' b'c' b'a' b'n' b' ' b'y' b'o' b'u' b' ' b'p' b'l'
 b'e' b'a' b's' b'e' b' ' b'p' b'a' b'i' b'n' b't' b' ' b't' b'h' b'e'
 b' ' b'f' b'e'], shape=(101,), dtype=string)


2023-03-28 08:02:30.858108: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [49118381]
	 [[{{node Placeholder/_0}}]]


In [19]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'I hate how you cant even say black paint anymore\nNow I have to say "Leroy can you please paint the fe'
b'nce?"\n\nWhat\'s the difference between a Jew in Nazi Germany and pizza ?\nPizza doesn\'t scream when you '
b"put it in the oven .  I'm so sorry.\n\nI recently went to America....\n...and being there really helped "
b'me learn about American culture. So I visited a shop and as I was leaving, the Shopkeeper said "Have '
b'a nice day!" But I didn\'t so I sued him.\n\nBrian raises his hand and says,  He s in Heaven. \nA Sunday '


2023-03-28 08:02:31.235720: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [49118381]
	 [[{{node Placeholder/_0}}]]


In [20]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [21]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [22]:
dataset = sequences.map(split_input_target)

In [23]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())


Input : b'I hate how you cant even say black paint anymore\nNow I have to say "Leroy can you please paint the f'
Target: b' hate how you cant even say black paint anymore\nNow I have to say "Leroy can you please paint the fe'


2023-03-28 08:02:34.680826: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [49118381]
	 [[{{node Placeholder/_0}}]]


In [24]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [25]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 32

In [26]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [27]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [28]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

2023-03-28 08:02:38.965959: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [49118381]
	 [[{{node Placeholder/_0}}]]
2023-03-28 08:02:38.966322: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [49118381]
	 [[{{node Placeholder/_0}}]]


(64, 100, 101) # (batch_size, sequence_length, vocab_size)


In [29]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  25856     
                                                                 
 gru (GRU)                   multiple                  27840     
                                                                 
 dense (Dense)               multiple                  3333      
                                                                 
Total params: 57,029
Trainable params: 57,029
Non-trainable params: 0
_________________________________________________________________


In [30]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [31]:
sampled_indices

array([ 78, 100,  66,  32,  11,  57,  87,  88,  29,  76,  51,  61,  33,
        59,  27,  84,  84,  60,  10,  60,  66,  77,   8,  88,  26,  47,
        41,  60,  87,   8,  32,  78,  66,  99,  45,  72,  43,   2,  21,
        70,  78,   8,  13,  93,  71,  28,  99,   3,  40,  61,  86,  96,
        90,  16,  65,  84,  50,  20,  24,  73,  53,  82,  51,  92,  83,
         0,  74,   6,  43,   1,  78,  53,   1,  15,  50,  45,  68,  13,
        99,  50,  41,   6,  60,   6,  86,   1,  98,  51,  86,  44,   3,
        95,  95,  70,  23,  85,  16, 100,  18,  94])

In [32]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())


Input:
 b'ster, 1991."  Now furious, another student yelled, "Oh yeah? Suck this!"  Pedro jumped out of his ch'

Next Char Predictions:
 b'i\x7f];&Trs8gNX<V6ooW%W]h#s5JDWr#;i]~HcF\t0ai#(xb7~\nCXq{u+\\oM/3dPmNwn[UNK]e!F\x08iP\x08*MH_(~MD!W!q\x08}NqG\nzza2p+\x7f-y'


In [33]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [34]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 101)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.6150846, shape=(), dtype=float32)


In [35]:
tf.exp(example_batch_mean_loss).numpy()

100.996376

In [36]:
model.compile(optimizer='adam', loss=loss)

In [43]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

model.load_weights(checkpoint_dir + "/checkpoint")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

2023-03-28 08:17:31.791324: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open ./training_checkpoints/checkpoint: DATA_LOSS: not an sstable (bad magic number): perhaps your file is in a different file format and you need to use a different restore operator?


OSError: Unable to open file (file signature not found)

In [38]:
EPOCHS = 20

In [39]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20


2023-03-28 08:02:52.283820: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [49118381]
	 [[{{node Placeholder/_0}}]]
2023-03-28 08:02:52.284156: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int64 and shape [49118381]
	 [[{{node Placeholder/_0}}]]
2023-03-28 08:02:52.695941: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split

 171/7598 [..............................] - ETA: 7:03 - loss: 3.3119

KeyboardInterrupt: 